# Building an Image Classifier Deep Network applying Transfer Learning

In [ ]:
import os

s3endpoint = os.environ['s3endpoint']
s3bucket = os.environ['s3bucket_data']
mount_path = os.environ['working_dir']
s3accessKey = os.environ['aws_access_key']
s3secretKey = os.environ['aws_secret_key']

In [2]:
# import the required libraries
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tf.keras.utils
from PIL import Image
import numpy as np
import os
import base64
from io import BytesIO

2023-12-15 09:22:23.721158: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 09:22:24.735155: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-15 09:22:24.735277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-15 09:22:24.735289: W 

# Clean data in PVC

In [7]:
# mount_path = "/data/"
# mount_path = ""
images_path = mount_path+"images"
models_path = mount_path+"models"

os.system("rm -rf "+images_path)
os.system("rm -rf "+models_path)
# os.system("ls "+images_path)




0

# Data Retrieval from S3

In [8]:
import boto3
from botocore.client import Config

# s3endpoint='https://minio-api-test02.apps.rhods-internal.61tk.p1.openshiftapps.com',
# s3endpoint = "https://minio-api-demo-ai.apps.cluster-ssfg4.dynamic.opentlc.com"
# s3bucket = "data"

s3_client = boto3.client('s3',
        endpoint_url=s3endpoint,
        aws_access_key_id=s3accessKey,
        aws_secret_access_key=s3secretKey,
        # aws_access_key_id='ZpPJfDh82BVcKnCYdpKH',
        # aws_secret_access_key='cB6xKinAwdWxZh8gE3SwPKPPsBEnfakSMaipF1rz',
        config=Config(signature_version='s3v4'),
        region_name='us-east-1')


objects = s3_client.list_objects_v2(Bucket=s3bucket)


for obj in objects['Contents']:
    if not obj['Key'].endswith("/"):
        print(obj['Key'])
        # print(obj.key)
        if not os.path.exists(os.path.dirname(mount_path+obj['Key'])):
            os.makedirs(os.path.dirname(mount_path+obj['Key']))
        s3_client.download_file(s3bucket,obj['Key'], mount_path+obj['Key'])